In [1]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import re

In [19]:
# create generators for the training and test sets
np.random.seed(0)
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

data_dir = "C:/Users/Filip/Desktop/PW/2 semestr/Deep Learning/pro1"

datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

train_generator = datagen.flow_from_directory(
    data_dir + "/train",
    target_size=(32, 32),  # Resize images to this size
    batch_size=1,  # Number of images to load at each iteration
    class_mode="categorical",
)  # Type of classification ('binary' or 'categorical')

test_generator = datagen.flow_from_directory(
    data_dir + "/test",
    target_size=(32, 32),
    batch_size=20,
    class_mode="categorical",
)

Found 90000 images belonging to 10 classes.
Found 90000 images belonging to 10 classes.


In [3]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
def generator_to_data(generator, num_samples):
    # Determine the output shape for the data based on the generator properties
    data_shape = (num_samples,) + generator.image_shape
    labels_shape = (
        num_samples,
        generator.num_classes,
    )  # Adjust for one-hot encoded labels

    # Initialize the arrays with the correct shape
    data = np.zeros(data_shape)
    labels = np.zeros(labels_shape)

    # Batch counter
    i = 0
    for inputs_batch, labels_batch in generator:
        batch_size = inputs_batch.shape[
            0
        ]  # Actual batch size may be less than generator.batch_size
        data_index_start = i * generator.batch_size
        data_index_end = data_index_start + batch_size

        # Assign the batch data and labels to the main arrays
        data[data_index_start:data_index_end] = inputs_batch
        labels[data_index_start:data_index_end] = labels_batch

        i += 1  # Increment the batch counter
        if i * generator.batch_size >= num_samples:
            # Break the loop when enough samples have been processed
            break
    # Since labels might be in one-hot encoding (if class_mode='categorical'), convert them back to single digits if necessary
    if generator.class_mode == "categorical":
        labels = np.argmax(labels, axis=1)
    return data, labels

In [5]:
#Generate the data
num_train_samples = 90000
num_test_samples = 9000
# Convert generator data to arrays
training_images, training_labels = generator_to_data(train_generator, num_train_samples)
test_images, test_labels = generator_to_data(test_generator, num_test_samples)



In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load VGG16 pre-trained on ImageNet, without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base_model
base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Compile the model
model.compile(optimizer=Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(training_images,
    training_labels,
    epochs=25,
    validation_data=(test_images, test_labels))


58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/25
2813/2813 [==============================] - 33s 9ms/step - loss: 1.5450 - accuracy: 0.4451 - val_loss: 1.4686 - val_accuracy: 0.4747
Epoch 2/25
 472/2813 [====>.........................] - ETA: 19s - loss: 1.4275 - accuracy: 0.4887

KeyboardInterrupt: 

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load VGG16 pre-trained on ImageNet, without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Unfreeze the last few layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=True)

# Train the model
model.fit(datagen.flow(training_images, training_labels, batch_size=32),
          epochs=25,
          validation_data=(test_images, test_labels),
          callbacks=[reduce_lr, early_stop, checkpoint])


Epoch 1/25
2813/2813 [==============================] - 77s 25ms/step - loss: 1.4124 - accuracy: 0.4961 - val_loss: 1.2242 - val_accuracy: 0.5837 - lr: 1.0000e-04
Epoch 2/25
2813/2813 [==============================] - 67s 24ms/step - loss: 1.0895 - accuracy: 0.6189 - val_loss: 1.2060 - val_accuracy: 0.6230 - lr: 1.0000e-04
Epoch 3/25
2813/2813 [==============================] - 67s 24ms/step - loss: 0.9909 - accuracy: 0.6548 - val_loss: 1.1052 - val_accuracy: 0.6367 - lr: 1.0000e-04
Epoch 4/25
2813/2813 [==============================] - 65s 23ms/step - loss: 0.9219 - accuracy: 0.6793 - val_loss: 1.3594 - val_accuracy: 0.6187 - lr: 1.0000e-04
Epoch 5/25
2813/2813 [==============================] - 65s 23ms/step - loss: 0.8659 - accuracy: 0.6983 - val_loss: 1.1397 - val_accuracy: 0.6453 - lr: 1.0000e-04
Epoch 6/25
2813/2813 [==============================] - 64s 23ms/step - loss: 0.8182 - accuracy: 0.7168 - val_loss: 0.8733 - val_accuracy: 0.7073 - lr: 1.0000e-04
Epoch 7/25
2813/2813 [

In [8]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models

# Parameters
num_classes = 10  # Change as per your dataset
num_epochs = 10
data_dir = "/home/filip/pro1"  # Base directory for your dataset

# Load AlexNet
alexnet = models.alexnet(pretrained=True)
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Data Transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Data
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Training and Validation Loops
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, loss: {running_loss / len(train_loader)}')

    alexnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = alexnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on validation set: {100 * correct / total}%')

print('Finished Training')


/home/filip/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/filip/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/filip/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|████████████████████████████████████████████████████████████████████████████████| 233M/233M [00:18<00:00, 13.3MB/s]


KeyboardInterrupt: 

In [17]:
import os
import re
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
from torchvision.models import alexnet, AlexNet_Weights
from tqdm import tqdm

def save_model_with_version(model, base_path="models/ALEX/"):
    pattern = r"v(\d+)"  # Regex to find version numbers
    max_version = 0
    
    # Check for existing models and find the highest version number
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    for filename in os.listdir(base_path):
        match = re.search(pattern, filename)
        if match:
            version = int(match.group(1))
            max_version = max(max_version, version)
    
    # Increment the version for the new model
    new_version = max_version + 1
    model_path = f"{base_path}ALEX{new_version}.pth"
    
    # Save the model
    torch.save(model.state_dict(), model_path)
    print(f"Model saved as {model_path}")

# Parameters
num_classes = 10  # Update to the number of classes in your dataset
num_epochs = 10
data_dir = "/home/filip/pro1"  # Base directory for your dataset

# Load AlexNet
weights = AlexNet_Weights.IMAGENET1K_V1
alexnet_model = alexnet(weights=weights)
alexnet_model.classifier[6] = nn.Linear(alexnet_model.classifier[6].in_features, num_classes)

# Data Transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Data
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet_model.parameters(), lr=0.001, momentum=0.9)

FileNotFoundError: [WinError 3] System nie może odnaleźć określonej ścieżki: '/home/filip/pro1/train'

In [10]:
for epoch in range(num_epochs):
    alexnet_model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', unit='batch')
    
    for inputs, labels in train_progress_bar:
        optimizer.zero_grad()
        outputs = alexnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        train_progress_bar.set_postfix(loss=train_loss/train_total, accuracy=100.*train_correct/train_total)

    # Validation
    alexnet_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validate]', unit='batch')
    
    with torch.no_grad():
        for images, labels in val_progress_bar:
            outputs = alexnet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_progress_bar.set_postfix(loss=val_loss/val_total, accuracy=100.*val_correct/val_total)

    print(f'End of Epoch {epoch + 1}, Training Loss: {train_loss / train_total}, Training Accuracy: {100. * train_correct / train_total}, Validation Loss: {val_loss / val_total}, Validation Accuracy: {100. * val_correct / val_total}')

# Evaluate the model
alexnet_model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = alexnet_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(val_loader)
accuracy = 100 * correct / total
print(f"Test Loss: {test_loss}, Test Accuracy: {accuracy}%")

# Save the model
save_model_with_version(alexnet_model)


Epoch 1/10 [Validate]: 100%|███████████████████████| 2813/2813 [8:31:55<00:00, 10.92s/batch, accuracy=75.6, loss=0.0214]


End of Epoch 1, Training Loss: 0.026410604591833222, Training Accuracy: 69.57, Validation Loss: 0.02144854884068999, Validation Accuracy: 75.60777777777778


Epoch 2/10 [Validate]: 100%|█████████████████████████| 2813/2813 [18:06<00:00,  2.59batch/s, accuracy=78.2, loss=0.0193]


End of Epoch 2, Training Loss: 0.020225613204969302, Training Accuracy: 76.8788888888889, Validation Loss: 0.019274487319340308, Validation Accuracy: 78.17222222222222


Epoch 3/10 [Validate]: 100%|█████████████████████████| 2813/2813 [17:41<00:00,  2.65batch/s, accuracy=78.8, loss=0.0186]


...

End of Epoch 8, Training Loss: 0.009424223492874039, Training Accuracy: 89.02888888888889, Validation Loss: 0.01780273897930747, Validation Accuracy: 80.6211111111111


Epoch 9/10 [Validate]: 100%|█████████████████████████| 2813/2813 [22:30<00:00,  2.08batch/s, accuracy=80.5, loss=0.0191]


End of Epoch 9, Training Loss: 0.008344897169164485, Training Accuracy: 90.29666666666667, Validation Loss: 0.019092753187896598, Validation Accuracy: 80.52777777777777


Epoch 10/10 [Validate]: 100%|████████████████████████| 2813/2813 [12:20<00:00,  3.80batch/s, accuracy=81.1, loss=0.0188]


End of Epoch 10, Training Loss: 0.007296247864638766, Training Accuracy: 91.54222222222222, Validation Loss: 0.01880865217689732, Validation Accuracy: 81.13222222222223
Test Loss: 0.6017698883472303, Test Accuracy: 81.13222222222223%
Model saved as models/ALEX/ALEX1.pth


In [11]:
!pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.2 MB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.4/1.2 MB 3.9 MB/s eta 0:00:01
   ----------------------------- ---------- 0.9/1.2 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 6.1 MB/s eta 0:00:00


In [12]:
import os
import re
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
from torchvision.models import alexnet, AlexNet_Weights
from tqdm import tqdm

In [12]:
torch.save(alexnet_model.state_dict(), f'alexnet_epoch_{epoch}.pth')

In [14]:
num_classes = 10  # Update to the number of classes in your dataset
num_epochs = 15
weights = AlexNet_Weights.IMAGENET1K_V1
alexnet_model = alexnet(weights=weights)
alexnet_model.classifier[6] = nn.Linear(alexnet_model.classifier[6].in_features, num_classes)
saved_model_path = f'alexnet_epoch_9.pth'
alexnet_model.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [20]:
# Data Transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Data
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet_model.parameters(), lr=0.001, momentum=0.9)

In [15]:
alexnet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
num_epochs = 20

In [22]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    alexnet_model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', unit='batch')
    
    for inputs, labels in train_progress_bar:
        optimizer.zero_grad()
        outputs = alexnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        train_progress_bar.set_postfix(loss=train_loss/train_total, accuracy=100.*train_correct/train_total)
        
    train_losses.append(train_loss / train_total)
    train_accuracies.append(100. * train_correct / train_total)

    # Validation
    alexnet_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validate]', unit='batch')
    
    
    with torch.no_grad():
        for images, labels in val_progress_bar:
            outputs = alexnet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_progress_bar.set_postfix(loss=val_loss/val_total, accuracy=100.*val_correct/val_total)
            
        val_losses.append(val_loss / val_total)
        val_accuracies.append(100. * val_correct / val_total)

    print(f'End of Epoch {epoch + 1}, Training Loss: {train_loss / train_total}, Training Accuracy: {100. * train_correct / train_total}, Validation Loss: {val_loss / val_total}, Validation Accuracy: {100. * val_correct / val_total}')

# Evaluate the model
alexnet_model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = alexnet_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(val_loader)
accuracy = 100 * correct / total
print(f"Test Loss: {test_loss}, Test Accuracy: {accuracy}%")

# Save the model
save_model_with_version(alexnet_model)


Epoch 1/20 [Validate]: 100%|██████████| 2813/2813 [14:07<00:00,  3.32batch/s, accuracy=76.4, loss=0.0206]


End of Epoch 1, Training Loss: 0.026588940094245804, Training Accuracy: 69.63555555555556, Validation Loss: 0.02061356782735222, Validation Accuracy: 76.39555555555556


Epoch 2/20 [Validate]: 100%|██████████| 2813/2813 [13:45<00:00,  3.41batch/s, accuracy=77.9, loss=0.0194]


End of Epoch 2, Training Loss: 0.0203347673503889, Training Accuracy: 76.74777777777778, Validation Loss: 0.019351996461219257, Validation Accuracy: 77.88


Epoch 3/20 [Validate]: 100%|██████████| 2813/2813 [13:49<00:00,  3.39batch/s, accuracy=79.2, loss=0.0182]


...

End of Epoch 18, Training Loss: 0.0030329054667937775, Training Accuracy: 96.56444444444445, Validation Loss: 0.023099956043567727, Validation Accuracy: 81.66


Epoch 19/20 [Validate]: 100%|██████████| 2813/2813 [14:01<00:00,  3.34batch/s, accuracy=81.2, loss=0.0236]


End of Epoch 19, Training Loss: 0.0027685977276854425, Training Accuracy: 96.85888888888888, Validation Loss: 0.023616440180124067, Validation Accuracy: 81.24888888888889


Epoch 20/20 [Validate]: 100%|██████████| 2813/2813 [13:55<00:00,  3.37batch/s, accuracy=81.7, loss=0.023] 


End of Epoch 20, Training Loss: 0.002638745779651476, Training Accuracy: 97.00222222222222, Validation Loss: 0.022956294250175577, Validation Accuracy: 81.71333333333334
Test Loss: 0.7344708434112343, Test Accuracy: 81.71333333333334%
Model saved as models/ALEX/ALEX1.pth
